## Лекция 4 - Элементы функционального программирования в Python

- синтаксис функций
- LEGB
- лямбды
- filter, map, zip
- генераторы и итераторы
- декораторы
- модуль functools
- модуль itertools (is back)

In [1]:
# пример простейших функций
# (уже не ново, т.к. видели примеры в пакете algo)

def foo(x):
    return x + 10
   
magic_number = foo(63)
magic_number

73

In [2]:
def bar():
    print('Hello')
    
bar()    

Hello


In [3]:
# функция может возвращать все что угодно:
# удобно, т.к. можно вернуть несколько значений сразу
# (по сути кортеж)
def arithmetics(a, b):
    ''' функция возвращает сумму, разность и произведение чисел '''
    summa = a + b
    diff = a - b
    mult = a * b
    return summa, diff, mult

s = arithmetics(10, 15)
s

(25, -5, 150)

In [4]:
_, difference, _ = arithmetics(7, 9)
difference

-2

In [5]:
# Синтаксис *args, **kwargs:
def pretty_print(text, *args, **kwargs):
    # draw header
    for cnt in args:
        print(kwargs['fill'] * cnt)
        
    # print main text
    print(text)
    
    # draw footer
    if kwargs['footer']:
        for cnt in reversed(args):
            print(kwargs['fill'] * cnt)
    
    # show info
    print()        
    print('args:')
    print(args)
    print('kwargs:')
    print(kwargs)

    
pretty_print('Hello', 20, 10, 5, fill='~', footer=True)

~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
~~~~~
Hello
~~~~~
~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~

args:
(20, 10, 5)
kwargs:
{'fill': '~', 'footer': True}


In [6]:
param_list = [50, 25, 10]

# param_list сам собой не развернется в последовательность параметров
# это будет один параметр - список:
pretty_print('Hello!', param_list, fill='~', footer=False)

TypeError: can't multiply sequence by non-int of type 'list'

In [7]:
# чтобы список трактовался как список параметров, пишем так:
pretty_print('Hello!', *param_list, fill='~', footer=False)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~
Hello!

args:
(50, 25, 10)
kwargs:
{'fill': '~', 'footer': False}


In [8]:
def print_chars(char, count):
    print(char * count)
    
print_chars('X', 10)

XXXXXXXXXX


In [9]:
print_chars(count=5, char='*')

*****


In [10]:
# демонстрация нюанса (список-параметр создается только единожды)
def append_item(item, arr=[]):
    arr.append(item)
    return arr

# здксь без эксцессов:
print(append_item(42, [1,2,3]))
print(append_item(73, [1,2,3]))

[1, 2, 3, 42]
[1, 2, 3, 73]


In [11]:
# а здесь - с эксцессами:
print(append_item(42))
print(append_item(73))

[42]
[42, 73]


In [12]:
# корректный обход нюанса
def append_item(item, arr=None):
    if arr is None:
        arr = []
    arr.append(item)
 
    return arr

print(append_item(42))
print(append_item(73))

[42]
[73]


### Функция - это first class объект

In [13]:
# очень важно: функция - это first class объект!
dir(pretty_print)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
# немного поинстроспектируем:
print('Qualified name:', pretty_print.__qualname__)
print('Vars:', pretty_print.__code__.co_varnames)

Qualified name: pretty_print
Vars: ('text', 'args', 'kwargs', 'cnt')


In [15]:
# эквивалентно foo(10)
foo.__call__(10)

20

In [16]:
# таким образом, функции можно присваивать переменным:
func = foo
func(100)

110

In [17]:
# можно делать массив из функций
# и последовательно выполнять их:
import math

funcs = [foo, math.factorial]

for func in funcs:
    print(func.__qualname__, func(5))

foo 15
factorial 120


In [18]:
# можно передавать функцию в функцию:
def process_elementwise(seq, func):
    return [func(s) for s in seq]

words = ['Fundamentals', 'of', 'brainwashing...']
process_elementwise(words, str.__len__)

[12, 2, 15]

In [19]:
process_elementwise(words, str.upper)

['FUNDAMENTALS', 'OF', 'BRAINWASHING...']

In [20]:
nums = [1, -3, 5, -7, -9]
process_elementwise(nums, abs)

[1, 3, 5, 7, 9]

In [21]:
process_elementwise(nums, foo)

[11, 7, 15, 3, 1]

### LEGB (области видимости)

- L, Local — Names assigned in any way within a function (def or lambda)), and not declared global in that function.
- E, Enclosing function locals — Name in the local scope of any and all enclosing functions (def or lambda), from inner to outer.
- G, Global (module) — Names assigned at the top-level of a module file, or declared global in a def within the file.
- B, Built-in (Python) — Names preassigned in the built-in names module : open,range,SyntaxError,...

In [22]:
x = 73

def wrong_scope():
    print(x)
    x += 1
    
wrong_scope()    

UnboundLocalError: local variable 'x' referenced before assignment

In [23]:
x = 73

def scope_ok():
    global x
    print(x)
    x += 1
    
scope_ok()

73


In [24]:
x = 5
y = 13

def make_closure():
    x = 42
    y = 911
    print('Outer locals:', locals())
    
    def func():
        global x
        print('func x, y:', x, y)
        print('func locals:', locals())
        x += 1

    return func

func = make_closure()
func()

print(x, y)

Outer locals: {'x': 42, 'y': 911}
func x, y: 5 911
func locals: {'y': 911}
6 13


In [25]:
def make_closure():
    value = 0
    def get_next_value():
        nonlocal value
        value += 1
        return value
    return get_next_value

get_next = make_closure()

print(get_next())
print(get_next())

1
2


In [26]:
# Версия Python 2 (где нет nonlocal)

# In python 2 there is no easy way to modify the value in the enclosing scope; 
# usually this is simulated by having a mutable value, 
# such as a list with length of 1:

def make_closure():
    value = [0]
    def get_next_value():
        value[0] += 1
        return value[0]

    return get_next_value

get_next = make_closure()

print(get_next())
print(get_next())

1
2


In [27]:
b = 1

# for-loop vars leak into global namespace
for b in range(5):
    if b == 4:
        print(b, '(b in for-loop)')
        
print(b, '(b in global)')

4 (b in for-loop)
4 (b in global)


In [28]:
# However, in Python 3.x, we can use closures to prevent 
# the for-loop variable to cut into the global namespace

i = 1
print([i for i in range(5)])
print(i, '(i in global)')

[0, 1, 2, 3, 4]
1 (i in global)


### Лямбда-функции

In [29]:
def duration_string(seconds):
    return '{}:{:02d}'.format(seconds // 60, seconds % 60)

print(duration_string(411))
print(duration_string(120))

6:51
2:00


In [30]:
# небольшие функции можно объявлять "на ходу"
# это анонимные функции (лямбда-функции)

dur = lambda seconds: '{}:{:02d}'.format(seconds // 60, seconds % 60)

print(dur(411))
print(dur(120))

6:51
2:00


In [31]:
def create_multipliers():
    return [lambda x: i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

8
8
8
8
8


In [32]:
def create_multipliers():
    return [lambda x, i=i: i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

0
2
4
6
8


In [33]:
from functools import partial
from operator import mul

def create_multipliers():
    return [partial(mul, i) for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

0
2
4
6
8


In [34]:
# Пример: посчитаем сумму цифр числа в декларативном стиле:
n = 123
print(sum(map(int, str(n))))

6


In [35]:
# функция reduce() в Python 2 является встроенной
from functools import reduce

s = range(1, 5)
reduce(lambda x, y: x + y, s)

10

### Итераторы и генераторы:
- итератор - это концепция (любой объект, имеющий методы ```next()``` и ```__iter__()```)
- генератор - языковое средство (объект вокруг функции с ```yield```)

Любой генератор является итератором, но не наоборот.

In [36]:
def odd_iterator(x):
    for i in range(1, x, 2):
        yield i

for x in odd_iterator(10):
    print(x)

1
3
5
7
9


In [37]:
it = odd_iterator(10)
it

<generator object odd_iterator at 0x02F3CC60>

In [38]:
print(next(it))
print(next(it))

1
3


In [39]:
try:
    it.throw(ValueError('Iterator problems!'))
except ValueError as err:
    print('Caught:', err)

it.close()

Caught: Iterator problems!


In [40]:
next(it)

StopIteration: 

In [41]:
dir(it)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

In [42]:
# Но это еще не все:
# есть также метод send, который позволяет делать т.н. корутины
def coroutine_example():
    while True:
        x = yield
        print('Got new number:', x)
        
cor = coroutine_example()
next(cor)

cor.send(2)
cor.send(5)
cor.send(10)

cor.close()

Got new number: 2
Got new number: 5
Got new number: 10


In [43]:
# предыдущий пример был игрушечный, но в асинхронных фреймворках
# корутины являются важнейшим элементом, и в них можно писать
# концептуально что-то вроде этого (чтобы это работало, нужен
# постоянно крутящийся цикл, в котором данные асинхронно 
# получаются и передаются, подробнее - в лекции №11):

# здесь логика генерации данных для обработки
def number_provider():
    for i in range(1, 11):
        yield i

# здесь логика получения данных и их обработки
def number_processor():
    while True:
        x = yield from number_provider
        print('Got new number:', x)

In [44]:
# в Python 3.3 появилась конструкция yield from:

def odd_number_provider(n):
    for i in range(1, n, 2):
        yield i

def even_number_provider(n):
    for i in range(2, n, 2):
        yield i
        
def number_generator(n):
    yield from odd_number_provider(n)
    yield from even_number_provider(n)
    
for number in number_generator(10):
    print(number)

1
3
5
7
9
2
4
6
8


In [45]:
# еще пример генератора (генерирующего цифры числа)
def generate_digits(x):
    while x > 0:
        yield x % 10
        x = x // 10

for digit in generate_digits(5716):
    print(digit)

6
1
7
5


In [46]:
l = [x for x in generate_digits(54321)]
l

[1, 2, 3, 4, 5]

In [47]:
i = generate_digits(4523)
print([next(i), next(i), next(i)])

[3, 2, 5]


In [48]:
# еще пример генератора (генерирующего перестановки n чисел)
import itertools

def permute(n):
    for perm in itertools.permutations(range(1, n+1)):
        yield perm
        
for p in permute(3):
    print(p)

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)


In [49]:
# еще раз о разнице между range в python2 и python3
nums = range(1, 5)
nums

# в Python2 будет выведено [1, 2, 3, 4]
# аналог: nums = xrange(1, 5)

range(1, 5)

In [50]:
# range - специфический итерируемый объект
# с дополнительными возможностями (например, индексирование)
dir(nums)

['__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

### Декораторы

In [51]:
def decorate(f):
    def new(*args, **kwargs):
        print(" ======= BEGIN ")
        f(args[0])
        print(" ======= END ")

    # декоратор возвращает функцию (callable)
    return new

# декорируем функцию
@decorate
def hello(x):
    print(x)

hello("OK")

# происходит такой вызов:
# decorate(hello("OK"))()

 ======= BEGIN 
OK
 ======= END 
